#### 1. 데이터 준비

다운로드 받은 데이터를 다음과 같이 .h5 파일들과 metadata.csv 파일이 동일한 위치에 저장되어 있도록 해주세요.

SPH (directory name)
- A00001.h5
- A00002.h5
- ....
- metadata.csv

#### 2. 전처리에 필요한 패키지 import

In [ ]:
import h5py
import numpy as np
import pandas as pd

from glob import glob 
from tqdm import tqdm 

#### 3. 파일 리스트 불러오기

위의 '데이터 준비'에서 안내드린대로 SPH 폴더 내에 .h5 파일들을 저장해놓고, glob을 이용하여 .h5 파일 리스트를 모두 가져오겠습니다.

아래 코드에 sorted를 이용하여 파일명을 정렬하였는데, 이 부분은 필수로 들어가야 하는 부분입니다. 반드시 포함하여 실행해주세요.

In [ ]:
filenames = sorted(glob("/DATA4/afib-renew/raw/SPH/*.h5"))

아래와 같이 h5 파일명 리스트를 가져왔다면 성공적으로 실행한 것입니다.

In [ ]:
filenames

#### 4. metadata 읽기

마찬가지로 동일한 폴더에 저장되어 있는 metadata.csv 파일을 pandas를 이용하여 읽습니다.

In [ ]:
df = pd.read_csv("/DATA4/afib-renew/raw/SPH/metadata.csv")

dataframe으로 읽은 뒤, 출력을 해보면 아래와 같이 나오는 것을 확인할 수 있습니다.

In [ ]:
df

dataframe에 'filename'이라는 컬럼을 추가하여, glob으로 읽어온 파일명들을 아래와 같이 저장하겠습니다.

In [ ]:
df['filename'] = filenames

위와 같이 저장을 하면, 아래와 같이 filename 컬럼이 추가되고, 그 안에 파일 경로가 추가된 것을 확인할 수 있습니다.

**주의** : ECG_ID와 filename의 .h5 파일명이 일치하는지 꼭 확인해주세요.

위에서 sorted로 정렬했던 이유가 바로 ECG_ID와 filename을 일치시키기 위함이었습니다.

In [ ]:
df

#### 5. 신호 데이터 읽기

예시로 첫번째 데이터를 읽는 방법을 안내드리겠습니다.

아래 코드를 참조해주세요.

In [ ]:
filename = df.filename.tolist()[0]

with h5py.File(filename, 'r') as f:
    signal = f['ecg'][:].T

위와 같이 데이터를 읽으면, 아래와 같이 5000x12 형태의 신호를 추출하게 됩니다.

In [ ]:
signal.shape

아시다시피, 우리는 Lead I 을 사용할 것입니다. 

따라서, 5000x12 shape의 array에서 Lead I에 해당되는 5000개의 데이터만 저장하면 됩니다. 

예제는 다음과 같습니다.

In [ ]:
lead1 = signal[:, 0]

#### 데이터 읽을 때 주의사항 + 추가 구현해야 할 부분

SPH 데이터는 길이가 가변적입니다.

10초 데이터만 있는 것이 아니라, 더 긴 신호가 저장되어 있을 수 있습니다.

따라서, 10초보다 긴 데이터의 경우 랜덤하게 10초를 샘플링하여 저장해주시면 됩니다.

아래와 같이 샘플 코드를 이용하시면 되겠습니다.

In [ ]:
# 데이터의 길이가 정확히 5000개인 경우
if lead1.shape[0] == 5000:
    lead1 = lead1 # 그대로 저장
# 데이터의 길이가 5000보다 큰 경우, 연속적인 5000개의 데이터를 랜덤하게 선택
else:
    start_idx = np.random.randint(0, lead1.shape[0] - 4999)  # 마지막 5000개를 포함할 수 있도록 인덱스 설정
    lead1 = lead1[start_idx:start_idx+5000, :]

위의 방법들을 이용하여, 전체 신호 데이터를 추출하여 하나의 array로 저장해주시면 됩니다.

#### 6. Labeling 방법 안내

처음에 읽었던 dataframe으로 돌아가서 AHA_Code라는 컬럼을 살펴보겠습니다.

In [ ]:
df.AHA_Code

위와 같이 숫자와 ';' 구분자 형태가 합쳐져 string으로 저장되어 있는 것을 확인할 수 있습니다.

labeling 방법은 간단합니다. 각 row를 string으로 형변환을 한 다음, '50' 이라는 숫자가 있으면 1, 없으면 0으로 저장해주시면 됩니다.

코드는 아래와 같습니다.

In [ ]:
# df['label'] = df['AHA_Code'].apply(lambda x: 1 if '50' in str(x) else 0)
# 'AHA_Code' 컬럼의 각 값에 대해 '50'이 포함되어 있는지 확인하고, 'label' 컬럼 생성
def check_contains_50(code):
    # 공백을 없애고, ';' 또는 '+'로 분리
    numbers = code.replace(' ', '').replace('+', ';').split(';')
    # '50'이 포함되어 있는지 확인
    return '50' in numbers
# 'label' 컬럼 생성
df['label'] = df['AHA_Code'].apply(check_contains_50).astype(int)

성공적으로 labeling을 마쳤습니다.

한번, labeling된 데이터의 개수를 출력해보겠습니다.

In [ ]:
df.label.value_counts()

Non-AFIB이 25,095개, AFIB이 675개로 출력된 것을 확인하였다면, 제대로 labeling이 된 것입니다.

#### 7. 데이터 저장

추출한 신호 데이터와 정답 데이터를 팀원분들께서 사용하시기 편한 형태로 저장합니다.

In [ ]:
# 데이터 저장 샘플 코드 (다른 데이터 형태로 저장하셔도 무방합니다. 편한 방법을 선택하세요.)
# np.save("/DATA4/afib-renew/processed/SPH-X.npy", X)
# np.save("/DATA4/afib-renew/processed/SPH-Y.npy", Y)